In [5]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [3]:
r = requests.get('https://www.themalaysianinsight.com/bahasa')

In [8]:
soup = BeautifulSoup(r.content, "lxml")
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if a_.get('href')]

In [13]:
topics = list(set([a_ for a_ in a if 'https://www.themalaysianinsight.com/bahasa/c' in a_]))
topics

['https://www.themalaysianinsight.com/bahasa/c/other-news',
 'https://www.themalaysianinsight.com/bahasa/c/editorial',
 'https://www.themalaysianinsight.com/bahasa/c/voices',
 'https://www.themalaysianinsight.com/bahasa/c/news',
 'https://www.themalaysianinsight.com/bahasa/c/pictures',
 'https://www.themalaysianinsight.com/bahasa/c/notices',
 'https://www.themalaysianinsight.com/bahasa/c/your-insight',
 'https://www.themalaysianinsight.com/bahasa/c/videos',
 'https://www.themalaysianinsight.com/bahasa/c/most-read']

In [69]:
topic_c = '-'.join(topics[0].split('/')[-3:])

In [74]:
def crawl(url, filename):
    if os.path.exists(filename):
        return True
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    divs = soup.find_all('div', {'class': 'story'})
    urls = []
    for div in divs:
        a = div.find_all('a')
        a = [a_.get('href') for a_ in a if a_.get('href')]
        a = list(set([a_ for a_ in a if 'https://www.themalaysianinsight.com/' in a_]))
        urls.extend(a)
    with open(filename, 'w') as fopen:
        json.dump(urls, fopen)
    return len(urls) > 0

In [71]:
directory = 'url'
!mkdir {directory}

In [82]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

for topic in topics:
    if 'most-read' in topic:
        continue
    topic_c = '-'.join(topic.split('/')[-3:])
    print(topic_c)
    done = False
    for i in tqdm(range(0, 100000, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [(f'{topic}?page={a}', 
                 os.path.join(directory, f'{topic_c}-{a}.json')) for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url[0], url[1]): url for url in urls}
        
        
        for future in as_completed(futures):
            if not future.result():
                done = True
        
        if done:
            break

bahasa-c-other-news


  1%|▌                                      | 131/10000 [00:29<36:28,  4.51it/s]


bahasa-c-editorial


  0%|                                      | 2/10000 [00:22<31:28:45, 11.33s/it]


bahasa-c-voices


...

  0%|                                                 | 0/10000 [00:21<?, ?it/s]


bahasa-c-your-insight


  0%|                                     | 10/10000 [01:41<28:09:02, 10.14s/it]


bahasa-c-videos


  0%|                                      | 32/10000 [00:15<1:20:31,  2.06it/s]
